In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))



In [2]:
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sqlite3 import Error
from sklearn.ensemble import RandomForestClassifier
import sqlite3
import pickle
import nltk
nltk.download('stopwords')
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andronikos/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:


dataset = pd.read_csv('./train.csv')
dataset.head()



,Unnamed: 0,title,text,subject,date,label
0,0,Obama’s Delusion Continues In Vapid Address To...,Obama addressed the Nation In a nothing burger...,politics,"Dec 6, 2015",0
1,1,Huge blast in China's Ningbo city kills at lea...,BEIJING (Reuters) - A powerful explosion in a ...,worldnews,"November 26, 2017",1
2,2,Trump applauds senators for new healthcare ref...,WASHINGTON (Reuters) - President Donald Trump ...,politicsNews,"September 13, 2017",1
3,3,UAW chief says Clinton told him she would rene...,(Reuters) - United Auto Workers President Denn...,politicsNews,"July 26, 2016",1
4,4,The Las Vegas Mass Shooting – More to the Stor...,Shawn Helton 21st Century WireAlthough many ar...,US_News,"October 12, 2017",0


In [4]:


dataset.groupby('text').text.count().plot.bar(ylim=0)
plt.show()



ValueError: Image size of 381x247113 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 432x288 with 1 Axes>

In [5]:


nltk.download('stopwords')
stemmer = PorterStemmer()
words = stopwords.words("english")
dataset['cleaned'] = dataset['text'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andronikos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:


dataset.head()



,Unnamed: 0,title,text,subject,date,label,cleaned
0,0,Obama’s Delusion Continues In Vapid Address To...,Obama addressed the Nation In a nothing burger...,politics,"Dec 6, 2015",0,obama address nation in noth burger jumbl jarg...
1,1,Huge blast in China's Ningbo city kills at lea...,BEIJING (Reuters) - A powerful explosion in a ...,worldnews,"November 26, 2017",1,beij reuter a power explos chines citi south s...
2,2,Trump applauds senators for new healthcare ref...,WASHINGTON (Reuters) - President Donald Trump ...,politicsNews,"September 13, 2017",1,washington reuter presid donald trump applaud ...
3,3,UAW chief says Clinton told him she would rene...,(Reuters) - United Auto Workers President Denn...,politicsNews,"July 26, 2016",1,reuter unit auto worker presid denni william s...
4,4,The Las Vegas Mass Shooting – More to the Stor...,Shawn Helton 21st Century WireAlthough many ar...,US_News,"October 12, 2017",0,shawn helton st centuri wirealthough mani stil...


In [7]:


vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
final_features = vectorizer.fit_transform(dataset['cleaned']).toarray()
final_features.shape



(100, 1731)

In [8]:


from sklearn.linear_model import LogisticRegression
X = dataset['cleaned']
Y = dataset['label']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

pipeline = Pipeline([('vect', vectorizer),
                     ('chi',  SelectKBest(chi2, k=1200)),
                     ('clf', LogisticRegression(random_state=0))])

model = pipeline.fit(X_train, y_train)
with open('LogisticRegression.pickle', 'wb') as f:
    pickle.dump(model, f)

ytest = np.array(y_test)

# confusion matrix and classification report(precision, recall, F1-score)
print(classification_report(ytest, model.predict(X_test)))
print(confusion_matrix(ytest, model.predict(X_test)))



              precision    recall  f1-score   support

           0       0.82      0.90      0.86        10
           1       0.93      0.87      0.90        15

    accuracy                           0.88        25
   macro avg       0.87      0.88      0.88        25
weighted avg       0.88      0.88      0.88        25

[[ 9  1]
 [ 2 13]]


In [10]:


from sklearn.linear_model import LogisticRegression
X = dataset['cleaned']
Y = dataset['label']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

pipeline = Pipeline([('vect', vectorizer),
                     ('chi',  SelectKBest(chi2, k=1200)),
                     ('clf', LogisticRegression(random_state=0))])

model = pipeline.fit(X_train, y_train)
with open('LogisticRegression.pickle', 'wb') as f:
    pickle.dump(model, f)

ytest = np.array(y_test)

# confusion matrix and classification report(precision, recall, F1-score)
print(classification_report(ytest, model.predict(X_test)))
print(confusion_matrix(ytest, model.predict(X_test)))



              precision    recall  f1-score   support

           0       0.88      0.64      0.74        11
           1       0.76      0.93      0.84        14

    accuracy                           0.80        25
   macro avg       0.82      0.78      0.79        25
weighted avg       0.81      0.80      0.79        25

[[ 7  4]
 [ 1 13]]
